In [2]:
import pandas as pd
import numpy as np

# ==============================================================================
# DEFINIÇÕES GLOBAIS (Consistente com a sua EDA e seleção final)
# ==============================================================================

# Lista final de features que o modelo treinado espera, NA ORDEM CORRETA.
# Esta lista foi definida no final da sua EDA.
FEATURES_FINAIS_PARA_MODELO = [
    'total_issues_created_log',
    'issue_participation_log',
    'issue_resolution_rate_created_log',
    'total_review_activity_log',
    'merged_ratio_log',
    'sum_lines_added_log',
    'interval',
    'branches_created',
]

# Epsilon para evitar divisão por zero nos ratios
EPSILON = 1e-6

# Colunas a serem removidas inicialmente se existirem
COLS_TO_DROP_INITIAL = [
    "project_id",
    "group_id",
    "mention_handle",
    "last_minute_commits", # Foi identificada como sempre zero
    "sum_lines_per_commit" # Foi removida devido à alta correlação/redundância
]

In [ ]:
# ==============================================================================
# FUNÇÃO DE PRÉ-PROCESSAMENTO
# ==============================================================================

def preprocess_data_for_prediction(df_new_raw: pd.DataFrame) -> pd.DataFrame:
    df_processed = df_new_raw.copy()
    print(f"Pré-processamento: Dimensões iniciais dos novos dados: {df_processed.shape}")

    # --- 1. Remover colunas irrelevantes ---
    print("Pré-processamento: Removendo colunas irrelevantes...")
    for col in COLS_TO_DROP_INITIAL:
        if col in df_processed.columns:
            df_processed.drop(columns=[col], inplace=True)
            print(f"  Coluna '{col}' removida.")
    print(f"Pré-processamento: Dimensões após remoção inicial: {df_processed.shape}")


    # --- 2. Criar features _log diretamente necessárias para o modelo ---
    # (Estas são features que entram no modelo como _log e não são derivadas de ratios complexos)
    print("Pré-processamento: Criando features _log diretas...")

    # sum_lines_added_log
    if 'sum_lines_added' in df_processed.columns:
        df_processed['sum_lines_added_log'] = np.log1p(df_processed['sum_lines_added'])
        print("  Feature 'sum_lines_added_log' criada.")
    else:
        print("  AVISO: Coluna 'sum_lines_added' não encontrada. 'sum_lines_added_log' não será criada diretamente aqui (será preenchida no final se faltar).")

    # total_issues_created_log
    if 'total_issues_created' in df_processed.columns:
        df_processed['total_issues_created_log'] = np.log1p(df_processed['total_issues_created'])
        print("  Feature 'total_issues_created_log' criada.")
    else:
        print("  AVISO: Coluna 'total_issues_created' não encontrada. 'total_issues_created_log' não será criada diretamente aqui.")

    # issue_participation_log
    if 'issue_participation' in df_processed.columns:
        df_processed['issue_participation_log'] = np.log1p(df_processed['issue_participation'])
        print("  Feature 'issue_participation_log' criada.")
    else:
        print("  AVISO: Coluna 'issue_participation' não encontrada. 'issue_participation_log' não será criada diretamente aqui.")


    # --- 3. Engenharia de Features (Ratios e Combinadas com Log) ---
    print("Pré-processamento: Iniciando engenharia de features (ratios, combinadas)...")

    # Merged Ratio Log
    if 'merged_requests' in df_processed.columns and 'total_merge_requests' in df_processed.columns:
        merged_ratio_raw = df_processed['merged_requests'] / (df_processed['total_merge_requests'] + EPSILON)
        df_processed['merged_ratio_log'] = np.log1p(merged_ratio_raw)
        print("  Feature 'merged_ratio_log' criada.")
    else:
        print("  AVISO: Colunas 'merged_requests' ou 'total_merge_requests' não encontradas. 'merged_ratio_log' não será criada (será preenchida no final se faltar).")
        df_processed['merged_ratio_log'] = np.log1p(0) # Preenchimento padrão se faltar

    # Issue Resolution Rate Created Log
    if 'issues_resolved' in df_processed.columns and 'total_issues_created' in df_processed.columns:
        issue_res_rate_created_raw = df_processed['issues_resolved'] / (df_processed['total_issues_created'] + EPSILON)
        df_processed['issue_resolution_rate_created_log'] = np.log1p(issue_res_rate_created_raw)
        print("  Feature 'issue_resolution_rate_created_log' criada.")
    else:
        print("  AVISO: Colunas 'issues_resolved' ou 'total_issues_created' não encontradas. 'issue_resolution_rate_created_log' não será criada (será preenchida no final se faltar).")
        df_processed['issue_resolution_rate_created_log'] = np.log1p(0) # Preenchimento padrão

    # Total Review Activity Log
    if 'review_comments_given' in df_processed.columns and 'review_comments_received' in df_processed.columns:
        total_review_raw = df_processed['review_comments_given'] + df_processed['review_comments_received']
        df_processed['total_review_activity_log'] = np.log1p(total_review_raw)
        print("  Feature 'total_review_activity_log' criada.")
    else:
        print("  AVISO: Colunas 'review_comments_given' ou 'review_comments_received' não encontradas. 'total_review_activity_log' não será criada (será preenchida no final se faltar).")
        df_processed['total_review_activity_log'] = np.log1p(0) # Preenchimento padrão

    # --- 4. Garantir que todas as features finais estão presentes e selecionar ---
    #    As features 'interval' e 'branches_created' são usadas como estão (se existirem).
    print("Pré-processamento: Selecionando e ordenando features finais...")
    missing_features_in_final_step = []
    for final_feature in FEATURES_FINAIS_PARA_MODELO:
        if final_feature not in df_processed.columns:
            print(f"  AVISO FINAL: Feature final '{final_feature}' não encontrada nos dados processados. Preenchendo com 0.")
            df_processed[final_feature] = 0 # Ou outra estratégia de imputação (ex: média do treino)
            missing_features_in_final_step.append(final_feature)
    
    if missing_features_in_final_step:
        print(f"  Resumo de features finais que foram preenchidas com 0: {missing_features_in_final_step}")

    # Selecionar apenas as colunas finais na ordem correta
    # Adiciona colunas faltantes com 0 se, mesmo após tudo, elas não existirem (caso extremo)
    for col in FEATURES_FINAIS_PARA_MODELO:
        if col not in df_processed:
            df_processed[col] = 0

    df_model_ready = df_processed[FEATURES_FINAIS_PARA_MODELO]
    print(f"Pré-processamento: Dimensões finais dos dados prontos para o modelo (antes do scaler): {df_model_ready.shape}")
    
    return df_model_ready


# Modelo